<a href="https://colab.research.google.com/github/LeandroCoelhos/estudos_datascience/blob/main/Lista06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import mplleaflet

In [ ]:
#!pip install mplleaflet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for mplleaflet: filename=mplleaflet-0.0.5-py3-none-any.whl size=28582 sha256=ca3809ddfa1560ef3bc085fdb8036de9dedce9da4fd26284aec675abccdaa5fd
  Stored in directory: /root/.cache/pip/wheels/6b/f5/21/cdd12e476182b4b0b98326cdb9efa02ddbd5d87ca5de051c84
Successfully built mplleaflet


In [ ]:
from google.colab import drive
drive.mount ('/drive')

Mounted at /drive


In [ ]:
df = pd.read_csv('/drive/MyDrive/MAE0399/Citi/201802-citibike-tripdata.csv',
                 sep = ',')

In [ ]:
estações = df['start station name'].unique()
cordenadas_cluster = df[(df['start station name'].isin(estações))][['start station name','start station latitude', 'start station longitude']].drop_duplicates()


In [ ]:
df.describe()

,tripduration,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,bikeid,birth year,gender
count,8.431140e+05,843114.000000,843114.000000,843114.000000,843114.000000,843114.000000,843114.000000,843114.000000,843114.000000,843114.000000
mean,8.773978e+02,1478.710958,40.737798,-73.983275,1465.978935,40.737472,-73.983562,26003.308082,1978.021175,1.167435
std,3.388293e+04,1405.478598,0.034001,0.017959,1402.289590,0.033708,0.018029,6399.367597,12.174693,0.484275
min,6.100000e+01,72.000000,40.646538,-74.017134,72.000000,40.646538,-74.041050,14529.000000,1885.000000,0.000000
25%,3.190000e+02,376.000000,40.719105,-73.994758,369.000000,40.719105,-73.995101,19432.000000,1969.000000,1.000000
50%,5.190000e+02,497.000000,40.739323,-73.986022,497.000000,40.739126,-73.986724,28384.000000,1980.000000,1.000000
75%,8.720000e+02,3172.000000,40.756687,-73.974987,3171.000000,40.756405,-73.975361,31657.000000,1988.000000,1.000000
max,1.596226e+07,3664.000000,45.506264,-73.568906,3668.000000,45.506264,-73.568906,33541.000000,2002.000000,2.000000


In [ ]:
df['cluster'] = '0'
col = 'start station latitude'
for i in range(10):
  perc = df[col].quantile(i/10)
  perc_prox = df[col].quantile((i+1)/10)
  aux = df[ (df[col] >= perc) & (df[col] < perc_prox)].index
  aux = aux.to_list()

  df.loc[aux, 'cluster'] = str(i)

In [ ]:
df['cluster_destino'] = '0'

In [ ]:
a = df['start station name'].unique()
b = df['end station name'].unique()

In [ ]:
c = list(a)
c.extend(list(b))
c = list(set(c))

In [ ]:
col_start = 'start station name'
col_fim = 'end station name'

In [ ]:
for i in range(len(c)):
  cl = df[df[col_start] == c[i]]['cluster'].unique()
  if (len(cl) == 0):
    cl = '11'
  else:
    cl = cl[0]

  aux = df[df[col_fim] == c[i]].index
  aux = aux.to_list()

  df.loc[aux, 'cluster_destino'] = cl

In [ ]:
df = df[df.cluster_destino != '11']
df

In [ ]:
origem = df['cluster'].value_counts()
origem

9    85774
6    85695
4    85361
2    84920
5    84381
7    84236
1    84017
0    83971
8    83206
3    81507
Name: cluster, dtype: int64

In [ ]:
destino = df['cluster_destino'].value_counts()
destino

6    87280
4    86727
5    85759
2    85658
1    84935
7    84794
8    83173
0    82866
3    82395
9    79481
Name: cluster_destino, dtype: int64

In [ ]:
cluster = ['0','1', '2', '3', '4', '5', '6', '7', '8', '9']

In [ ]:
a = []
for i in range(10):
  t = df[df['cluster'] == str(i)]['cluster_destino'].value_counts().sum()
  a.append(df[df['cluster'] == str(i)]['cluster_destino'].value_counts() / t)

b = pd.DataFrame(a, index = cluster)
b = b[cluster]
b

,0,1,2,3,4,5,6,7,8,9
0,0.813138,0.127473,0.024639,0.011754,0.007133,0.004787,0.006205,0.002930,0.001655,0.000286
1,0.119357,0.444255,0.227918,0.076127,0.042027,0.030779,0.026340,0.020317,0.008474,0.004404
2,0.022786,0.225330,0.322080,0.199058,0.098587,0.051978,0.037541,0.024341,0.011917,0.006382
3,0.010858,0.077846,0.205479,0.260836,0.222447,0.103574,0.065737,0.031494,0.015606,0.006122
4,0.005424,0.041764,0.099120,0.211923,0.255679,0.191247,0.111725,0.051569,0.022493,0.009056
5,0.005380,0.030730,0.055522,0.103210,0.199998,0.210723,0.232730,0.102547,0.042118,0.017042
6,0.004761,0.025859,0.038077,0.060062,0.114441,0.235031,0.221962,0.178272,0.094218,0.027318
7,0.003277,0.021143,0.026366,0.034202,0.052032,0.117028,0.193955,0.274467,0.214243,0.063286
8,0.001322,0.009446,0.012547,0.016489,0.026993,0.047797,0.103178,0.233637,0.340504,0.208086
9,0.000315,0.005654,0.007776,0.007566,0.011157,0.021370,0.033588,0.085282,0.234465,0.592825


#Maepando Com Geolocalização

In [ ]:
cordenadas_cluster
lons = cordenadas_cluster['start station longitude'].to_list()
lats = cordenadas_cluster['start station latitude'].to_list()

In [ ]:
fig = plt.figure(figsize=(20, 10))    #This is missing in your code.
plt.plot(lons, lats, 'r.',markerfacecolor='blue', markersize = 20)
mplleaflet.display(fig=fig)
#It will display the matplotlib object created by plot function